In [11]:
# Note: this slide imports a bunch of stuff for the peano-indexed Vec example
# Don't forget to run me before starting the presentation!!!
from __future__ import annotations
from typing import Any, Generic, TypeVar
from dataclasses import dataclass
N,T = TypeVar("N", bound="Nat"), TypeVar("T")

# Liquid Types, Revisited
## Nathan Taylor (ntaylor@cs), Sam Thomas (sgt@cs)

### C S 395T: The Model Checking Paradigm, 24 April 2023

Type theory is widely adopted in the PL community, and with good reason: We have strong metatheoretic guarantees around:

* Soundness: A type system will always reject an invalid program
* Automation: A type system (may!) require minimal intervention to execute
* Subtyping: Safely substituting one type for another
* Termination: Type-theoretic algorithms will always terminate with an answer, usually in small-polynomial time


## A type theory paper in a model-checking class?

However...  In this class we've talked about things like propositions, proofs, counterproofs... some might say that this feel like a different world than the typed programming you know from languages like Java?

Under the Curry-Howard correspondence, we _know_[^1] that:

* a type _is_ a logical proposition;
* a well-typed program _is_ a proof of each of its propositions;
  * a type error _is_ a counterproof of one or more of its propositions

Type semantics relates a class of inputs  (all possible valid envioronments) to a class of outputs (all possible instantiations of a type) - feels like abstraction or “modulo”!!!

[^1] At least to a first-order approximation.  Some terms and conditions may apply; see in store for details.

## Do well-typed programs never go wrong?

The point of a type system should be that it rejects invalid programs, but: can you think of input to `avg()` that will still cause an exception to be thrown?

In [3]:
def avg(xs: list[int]) -> int:
    """ Computes the average of a list of ints."""
    return sum(xs) // len(xs)

assert avg([1,2,3]) == 2
assert avg([1]) == 1


## Extending the expressivity of a type system

* Somebody in this room once said, "An abstraction is a constrained proof system"
    * a richer _constraint domain_ in our proof system lets us encode richer abstractions.
    * we may pay a cost for a richer _constraint domain_, however...

What sort of constraint domain would let us encode the type of a list that is provably nonempty (and thus avoiding the division by zero exception), while still maintaining soundness, automation, and termination?

---

Our goal: 
```python
def avg(xs: {TODO: proposition that rejects the empty list} ) -> int:
    return sum(xs) // len(xs)
avg([])      # should produce a type error
avg([1,2,3]) # should not produce a type error
```

## Non-solution 1: Keep the proof system; enrich the List type

A standard solution is to _index_ the List type with an additional type parameter which represents its length.

This means we need to teach _the type system_ about the natural numbers.

In [6]:
# An encoding of the natural numbers, entirely in the type system
# (notice the S and Z classes have no fields!)
class Nat: pass
class Z(Nat): pass              # The Z type is zero
class S(Nat, Generic[N]): pass  # The S<N> type is the successor of some other nat N

In [ ]:
# Note: `Two` and `Four` are typedefs for some subtypes of Nat, not 
# program-level terms with values we can compute on at runtime!
Two: type  = S[S[Z]]
Four: type = S[S[Two]]

The two constructors `nil` and `cons` enforce the invariant that the typelevel value of `N` always conforms to the length of the list.

We did this without needing to make the type system more expressive, and the benefits we already get from it remain!

In [ ]:
# Here's our vector of N elements, all of type T.
@dataclass
class Vec(Generic[N,T]):
    l: list[T]
        
def nil() -> Vec[Z, T]: return Vec([])
def cons(t: T, l: Vec[N, T]) -> Vec[S[N], T]: return Vec([t] + l.l)

# The type system knows the lengths of these lists!
empty: Vec[Z, int] = nil()
one_three:  Vec[Two, int] = cons(1, cons(3, nil()))

```python
def avg(l: Vec[S[N], int]) -> int:
    """ Produces the average of a vector of ints, whose length
    is the successor of some natural number (i.e. greater than 0!)"""
    return sum(l.l) // len(l.l)

avg(42)    # E: "Literal[42]" is incompatible with "Vec[S[N@avg], int]"
avg(nil()) # E: TypeVar "N@Vec" is covariant; "Z" is incompatible with "S[N@avg]"
avg(cons(1, nil())) # ok, as we'd hope
```

✓ This implementation of `avg` statically rejects the empty list!  This was the point of the exercise.

✓ All the existing benefits of our existing type system (soundness, automation, subtyping, termination) apply here, because we haven't made the metatheory more complicated

✗ However, counterproofs are opaque and difficulty to map back to the original program error.

## Non-solution 2: enrich the proof system; keep the List type

Previously, `Vec[N,T]` took two type-level arguments.  We could not parameterize it on a program term of type `int` (if we could, we could have avoided redefining the natural numbers in the type system.)

A type system that _does_ allow this is called a _dependent type system_.  

```haskell
constant vec : Type u → ℕ → Type u

-- notice that we can write 0 and (n + 1) directly! 
constant empty : Π α : Type u, vec α 0
constant cons :
  Π (α : Type u) (n : ℕ), α → vec α n → vec α (n + 1)
```

TODO: they are awesome except enjoy writing the proofs

## Non-solution 3: just use a model checker

"At no point is `avg()` passed the empty list" is a temporal safety property; any number of off-the-shelf tools (SLAM, BLAST, Houdini, etc...) or techniques from class could be leveraged here.  



✓ We've seen that model checking techniques are sound, push-button in their automation, and can scale to non-trivial problems, just like a good typechecker.


✓ Since model checking explores program states, it's sensitive to program control flow, leading to a more precise abstraction than a type-based approach.

✗ However... as we saw from other presentations, reasoning about inductive data structures (lists, trees, etc...) typically requires quantifiers and/or custom logics.  With types, quantifications are handled via polymorphism and recursive definitions pose no difficulty.

## Unifying the non-solutions with _liquid types_

A _refinement type_ pairs an ordinary type (the _base type_) with a _refining predicate_:

\begin{equation}
\{v: int\; | \; 0 \leq v\; ∧\; v < n\} 
\end{equation}

This particular type refines its inhabitants to be between 0 and some in-scope program variable `n`.  `n` is a term, so a refinement type is a dependent type, too!  

$v$ is the distinguished _value variable_, referring to whatever term the type is typing.

### If predicates are drawn from _SMT-decidable theories_, then we have a _logically-qualified (LiQuid) type system_.

```python
@solvent.check
def avg(xs: {len(V) > 0}) -> int
    return sum(xs) // len(xs)
avg(42) # Type-checking error
avg([]) # Type-checking error
```

In this talk, we will eludcidate the liquid type technique by way of our implementation of liquid types for Python (See https://github.com/dijkstracula/solvent)

Two classes of type meta-algorithms:

* Type _checking_: Verifies that a program fragment _annotated_ with types is consistent within the theory
* Type _reconstruction_: Given a program fragment _unannotated_ with types, infer a consistent set of types
  * generally the harder problem since it requires coming up with a consistent type, not just verifying it

Naturally, reconstruction is the problem this paper is interested in, setting the bar high!

## Reconstruction of a liquid type...

\begin{equation}
\{v: \text{<base type>} \;|\; \text{<predicate>} \wedge \text{<predicate>} \wedge \text{...} \} 
\end{equation}
    
Involves decomposing the problem into reconstructing the base type and the refinements: for each term to be typed, we perform:
    
1) Base type inference, using the venerable Hindley-Milner algorithm;
2) Liquid constraint generation, capturing subtyping relationships that must be satisfied;
3) Liquid constraint solving, finding a "good" conjunction of predicates that satisifies the constraints.
    
The key insight of the paper comes up at step 3!

## Which base type to reconstruct?

TODO: have a solvent decorator that stops at H-M constraint generation, and after unification/base type gen.  

In [5]:
from solvent.frontend import *

@infer_base_constraints
#@infer_base
def max(x, y):
    if x > y:
        return x
    else:
        return y

Constraints:    bool = bool; '9 = '10; '9 = int; '10 = int
Ununified type: ('9, '10) -> '9


## Which logical constraints to reconstruct?

```python
def max(x: int, y: int) -> int:
    if x > y:
        return x
    else:
        return y
```

Now that we have our base types figured out, what logical facts can we say about each particular int in this program?

TODO: It'd be nice, for expository purposes, to say something like "in H-M we begin by coming up with constraint variables for all the terms, and look we begin by doing the same thing again here" 

## Which refinement predicate to reconstruct?

```python
def max(x: int, y: int) -> { V: int | TODO: erm...what predicate should be chosen }
    if x > y:
        return x
    else:
        return y
print(max(42, 99))
```

\begin{equation}
\{v: int \;|\; true\}  \text{...probably too underspecified!}
\end{equation} 

\begin{equation}
\begin{split}
\{v: int \;|\; v >= x\; \wedge \; v >= y\}\; & \text{...seems like a good choice!} \\
\{v: int \;|\; v = x\; \vee \; v = y\}\; & \text{...overfits a bit!}
\end{split}
\end{equation}

\begin{equation}
\{v: int \;|\; v = 99\} \text{... certainly way too overspecified!}
\end{equation}

## Drawing from a fixed set of qualifiers

Because there are any number of possible qualifiers, a liquid type system _fixes_ a "basis set" of logical qualifiers patterns that predicates can be generated from.  The paper's worked examples requires only a small subset of the full set:

\begin{equation}
\begin{split}
0 & \leq v & \\
\star & \leq v   \; & \text{for $\star$ : int}     \\
v & \lt \star     \; & \text{for $\star$ : int}    \\
v & \lt len(\star) \; & \text{for $\star$ : array} \\
\end{split}
\end{equation}

Recall that $v$ is the distinguished _value variable_, referring to whatever term the type is typing.

$\star$ is a special _placeholder_ that well-typed program terms can be substituted in for.

Something _not_ discussed in the paper is that these qualifiers can be overriden by the programmer based on what qualifiers they expect to be useful for their program.  

For instance, the liquid type `Vec` implementation comes with an additional set of qualifiers, written by the implementer of the data structure:

```ocaml
(* a portion of ./postests/vec/len.hquals, simplified *)
qualif LVAR(v): len(v) [<, =, >] *
qualif LCONST(v): len(v) [<, =, >]  [0, 1]
qualif SETAPPEND(v): len(v) = (len(v) > * ? len(v) : * + 1)
qualif LSUM(v) : len(v) = [0, 1] + len( * ) + len( * )
qualif TOARR(v): len( * ) = Array.length v
```


* both the type system implementer, and the application programmer, can _bias_ the inferred types.  
* Because the set of qualifiers is always finite, we have constraint generation termination guarantees.